<div style="width:100%; height:140px">
    <img src="https://www.kuleuven.be/internationaal/thinktank/fotos-en-logos/ku-leuven-logo.png/image_preview" width = 300px, heigh = auto align=left>
</div>


KUL H02A5a Computer Vision: Group Assignment 2
---------------------------------------------------------------
Student numbers: <span style="color:red">r0708518, r0927391, r0925509, r0924356, r0912639</span>. (fill in your student numbers!)

In this group assignment your team will delve into some deep learning applications for computer vision. The assignment will be delivered in the same groups from *Group assignment 1* and you start from this template notebook. The notebook you submit for grading is the last notebook you submit in the [Kaggle competition](https://www.kaggle.com/t/d11be6a431b84198bc85f54ae7e2563f) prior to the deadline on **Tuesday 24 May 23:59**. Closely follow [these instructions](https://github.com/gourie/kaggle_inclass) for joining the competition, sharing your notebook with the TAs and making a valid notebook submission to the competition. A notebook submission not only produces a *submission.csv* file that is used to calculate your competition score, it also runs the entire notebook and saves its output as if it were a report. This way it becomes an all-in-one-place document for the TAs to review. As such, please make sure that your final submission notebook is self-contained and fully documented (e.g. provide strong arguments for the design choices that you make). Most likely, this notebook format is not appropriate to run all your experiments at submission time (e.g. the training of CNNs is a memory hungry and time consuming process; due to limited Kaggle resources). It can be a good idea to distribute your code otherwise and only summarize your findings, together with your final predictions, in the submission notebook. For example, you can substitute experiments with some text and figures that you have produced "offline" (e.g. learning curves and results on your internal validation set or even the test set for different architectures, pre-processing pipelines, etc). We advise you to first go through the PDF of this assignment entirely before you really start. Then, it can be a good idea to go through this notebook and use it as your first notebook submission to the competition. You can make use of the *Group assignment 2* forum/discussion board on Toledo if you have any questions. Good luck and have fun!

---------------------------------------------------------------
NOTES:
* This notebook is just a template. Please keep the five main sections, but feel free to adjust further in any way you please!
* Clearly indicate the improvements that you make! You can for instance use subsections like: *3.1. Improvement: applying loss function f instead of g*.


# 1. Overview
This assignment consists of *three main parts* for which we expect you to provide code and extensive documentation in the notebook:
* Image classification (Sect. 2)
* Semantic segmentation (Sect. 3)
* Adversarial attacks (Sect. 4)

In the first part, you will train an end-to-end neural network for image classification. In the second part, you will do the same for semantic segmentation. For these two tasks we expect you to put a significant effort into optimizing performance and as such competing with fellow students via the Kaggle competition. In the third part, you will try to find and exploit the weaknesses of your classification and/or segmentation network. For the latter there is no competition format, but we do expect you to put significant effort in achieving good performance on the self-posed goal for that part. Finally, we ask you to reflect and produce an overall discussion with links to the lectures and "real world" computer vision (Sect. 5). It is important to note that only a small part of the grade will reflect the actual performance of your networks. However, we do expect all things to work! In general, we will evaluate the correctness of your approach and your understanding of what you have done that you demonstrate in the descriptions and discussions in the final notebook.

## 1.1 Deep learning resources
If you did not yet explore this in *Group assignment 1 (Sect. 2)*, we recommend using the TensorFlow and/or Keras library for building deep learning models. You can find a nice crash course [here](https://colab.research.google.com/drive/1UCJt8EYjlzCs1H1d1X0iDGYJsHKwu-NO).

In [1]:
import cv2
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
import numpy as np
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt
from PIL import Image
from tqdm.notebook import tqdm
import winsound
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

## 1.2 PASCAL VOC 2009
For this project you will be using the [PASCAL VOC 2009](http://host.robots.ox.ac.uk/pascal/VOC/voc2009/index.html) dataset. This dataset consists of colour images of various scenes with different object classes (e.g. animal: *bird, cat, ...*; vehicle: *aeroplane, bicycle, ...*), totalling 20 classes.

In [2]:
# Loading the training data
train_df = pd.read_csv('D:/computer_vision/group_assignments/data_ga2/train/train_set.csv', index_col="Id")
labels = train_df.columns
train_df["img"] = [np.load('D:/computer_vision/group_assignments/data_ga2/train/img/train_{}.npy'.format(idx)) for idx, _ in train_df.iterrows()]
train_df["seg"] = [np.load('D:/computer_vision/group_assignments/data_ga2/train/seg/train_{}.npy'.format(idx)) for idx, _ in train_df.iterrows()]
print("The training set contains {} examples.".format(len(train_df)))

# Show some examples
# fig, axs = plt.subplots(2, 20, figsize=(10 * 20, 10 * 2))
# for i, label in enumerate(labels):
#     df = train_df.loc[train_df[label] == 1]
#     axs[0, i].imshow(df.iloc[0]["img"], vmin=0, vmax=255)
#     axs[0, i].set_title("\n".join(label for label in labels if df.iloc[0][label] == 1), fontsize=40)
#     axs[0, i].axis("off")
#     axs[1, i].imshow(df.iloc[0]["seg"], vmin=0, vmax=20)  # with the absolute color scale it will be clear that the arrays in the "seg" column are label maps (labels in [0, 20])
#     axs[1, i].axis("off")
#
# plt.show()
#
# The training dataframe contains for each image 20 columns with the ground truth classification labels and 20 column with the ground truth segmentation maps for each class
# train_df.head(1)

The training set contains 749 examples.


In [3]:
# Loading the test data
test_df = pd.read_csv('D:/computer_vision/group_assignments/data_ga2/test/test_set.csv', index_col="Id")
test_df["img"] = [np.load('D:/computer_vision/group_assignments/data_ga2/test/img/test_{}.npy'.format(idx)) for idx, _ in test_df.iterrows()]
test_df["seg"] = [-1 * np.ones(img.shape[:2], dtype=np.int8) for img in test_df["img"]]
print("The test set contains {} examples.".format(len(test_df)))
#
# # The test dataframe is similar to the training dataframe, but here the values are -1 --> your task is to fill in these as good as possible in Sect. 2 and Sect. 3; in Sect. 6 this dataframe is automatically transformed in the submission CSV!
# test_df.head(1)

The test set contains 750 examples.


## 1.3 Your Kaggle submission
Your filled test dataframe (during Sect. 2 and Sect. 3) must be converted to a submission.csv with two rows per example (one for classification and one for segmentation) and with only a single prediction column (the multi-class/label predictions running length encoded). You don't need to edit this section. Just make sure to call this function at the right position in this notebook.

In [ ]:
def _rle_encode(img):
    """
    Kaggle requires RLE encoded predictions for computation of the Dice score (https://www.kaggle.com/lifa08/run-length-encode-and-decode)

    Parameters
    ----------
    img: np.ndarray - binary img array
    
    Returns
    -------
    rle: String - running length encoded version of img
    """
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    rle = ' '.join(str(x) for x in runs)
    return rle

def generate_submission(df):
    """
    Make sure to call this function once after you completed Sect. 2 and Sect. 3! It transforms and writes your test dataframe into a submission.csv file.
    
    Parameters
    ----------
    df: pd.DataFrame - filled dataframe that needs to be converted
    
    Returns
    -------
    submission_df: pd.DataFrame - df in submission format.
    """
    df_dict = {"Id": [], "Predicted": []}
    for idx, _ in df.iterrows():
        df_dict["Id"].append(f"{idx}_classification")
        df_dict["Predicted"].append(_rle_encode(np.array(df.loc[idx, labels])))
        df_dict["Id"].append(f"{idx}_segmentation")
        df_dict["Predicted"].append(_rle_encode(np.array([df.loc[idx, "seg"] == j + 1 for j in range(len(labels))])))
    
    submission_df = pd.DataFrame(data=df_dict, dtype=str).set_index("Id")
    submission_df.to_csv("submission.csv")
    return submission_df

# 2. Image classification
The goal here is simple: implement a classification CNN and train it to recognise all 20 classes (and/or background) using the training set and compete on the test set (by filling in the classification columns in the test dataframe).

In [ ]:
class RandomClassificationModel:
    """
    Random classification model: 
        - generates random labels for the inputs based on the class distribution observed during training
        - assumes an input can have multiple labels
    """
    def fit(self, X, y):
        """
        Adjusts the class ratio variable to the one observed in y. 

        Parameters
        ----------
        X: list of arrays - n x (height x width x 3)
        y: list of arrays - n x (nb_classes)

        Returns
        -------
        self
        """
        self.distribution = np.mean(y, axis=0)
        print("Setting class distribution to:\n{}".format("\n".join(f"{label}: {p}" for label, p in zip(labels, self.distribution))))
        return self
        
    def predict(self, X):
        """
        Predicts for each input a label.
        
        Parameters
        ----------
        X: list of arrays - n x (height x width x 3)
            
        Returns
        -------
        y_pred: list of arrays - n x (nb_classes)
        """
        np.random.seed(0)
        return [np.array([int(np.random.rand() < p) for p in self.distribution]) for _ in X]
    
    def __call__(self, X):
        return self.predict(X)
    
model = RandomClassificationModel()
model.fit(train_df["img"], train_df[labels])
test_df.loc[:, labels] = model.predict(test_df["img"])
test_df.head(1)

## 2.0. Utilities

In [4]:
models_path = "../data_ga2/models/"

In [5]:
def reshape_images(images, size):
    new_images = []
    for img in images:
        resized = cv2.resize(img, (size, size), interpolation=cv2.INTER_AREA)
        new_images.append(resized)
    return np.array(new_images)

In [6]:
def make_sound():
    try:
        winsound.Beep(400, 750)
        winsound.Beep(500, 750)
        winsound.Beep(700, 750)
    except:
        pass

## 2.1. MobileNetV2 (0.94/0.957/0.9277)
### 2.1.1. Transfer learning (0.94)

In [9]:
X_train = reshape_images(train_df["img"], 224)
y_train = np.asarray(train_df[train_df.columns[:20]])
X_test = reshape_images(test_df["img"], 224)
print("Shape of X_train images is {} - Shape of X_test images is {} - Shape of y_train labels is {}".format(X_train[0].shape, X_test[0].shape, y_train.shape))

Shape of X_train images is (224, 224, 3) - Shape of X_test images is (224, 224, 3) - Shape of y_train labels is (749, 20)


In [10]:
class MobileNetV2_Model:

    def __init__(self):
        mobv2_model = tf.keras.applications.MobileNetV2(include_top=False, input_shape=(224,224,3), weights="imagenet")
        for l in mobv2_model.layers:
            l.trainable = False
        new_l = tf.keras.layers.Flatten()(mobv2_model.output)
        new_l = tf.keras.layers.Dense(20, activation="sigmoid")(new_l)
        self.model = tf.keras.Model(inputs=mobv2_model.input, outputs=new_l)

    def fit(self, X, y):
        X_train = tf.keras.applications.mobilenet_v2.preprocess_input(X)
        self.model.compile(run_eagerly=True, optimizer=tf.keras.optimizers.Adam(1e-5), loss='binary_crossentropy', metrics=['binary_accuracy'])
        callbacks = [tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3, min_delta=0.01),
                     tf.keras.callbacks.ModelCheckpoint(filepath="../../data_ga2/models/mobv2_m1_best.h5", monitor="val_loss", save_best_only=True)]
        self.model.fit(X_train, y, epochs=20,
                       batch_size=32, validation_split=0.2, callbacks=callbacks)

    def predict(self, X):
        return self.model.predict(tf.keras.applications.mobilenet_v2.preprocess_input(X), verbose=0)

    def __call__(self, X):
        return self.predict(X)

In [97]:
mobv2_m1 = MobileNetV2_Model()
make_sound()
mobv2_m1.model.summary()

Model: "model_15"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_16 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_16[0][0]']               
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                          

In [98]:
history_mobv2_m1 = mobv2_m1.fit(X_train, y_train)
make_sound()

Epoch 1/20
19/19 [==============================] - 40s 2s/step - loss: 0.5705 - binary_accuracy: 0.7407 - val_loss: 0.3411 - val_binary_accuracy: 0.8830
Epoch 2/20
19/19 [==============================] - 39s 2s/step - loss: 0.3007 - binary_accuracy: 0.9129 - val_loss: 0.2990 - val_binary_accuracy: 0.9143
Epoch 3/20
19/19 [==============================] - 42s 2s/step - loss: 0.2625 - binary_accuracy: 0.9254 - val_loss: 0.2780 - val_binary_accuracy: 0.9203
Epoch 4/20
19/19 [==============================] - 52s 3s/step - loss: 0.2279 - binary_accuracy: 0.9319 - val_loss: 0.2596 - val_binary_accuracy: 0.9237
Epoch 5/20
19/19 [==============================] - 45s 2s/step - loss: 0.1973 - binary_accuracy: 0.9383 - val_loss: 0.2425 - val_binary_accuracy: 0.9253
Epoch 6/20
19/19 [==============================] - 47s 2s/step - loss: 0.1708 - binary_accuracy: 0.9451 - val_loss: 0.2282 - val_binary_accuracy: 0.9280
Epoch 7/20
19/19 [==============================] - 53s 3s/step - loss: 0.14

### 2.1.2. Transfer learning + fine-tuning (0.957)

In [106]:
class MobileNetV2_fine_Model:

    def __init__(self, X, y):
        base_model = tf.keras.applications.MobileNetV2(include_top=False, input_shape=(224,224,3), weights="imagenet")
        base_model.trainable = False

        new_l = tf.keras.layers.Flatten()(base_model.output)
        new_l = tf.keras.layers.Dense(20, activation="sigmoid")(new_l)
        model = tf.keras.Model(base_model.input, new_l)

        X_train = tf.keras.applications.mobilenet_v2.preprocess_input(X)

        model.compile(run_eagerly=True, optimizer=tf.keras.optimizers.Adam(1e-4), loss='binary_crossentropy', metrics=['binary_accuracy'])
        callbacks = [tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3, min_delta=0.01),
                     tf.keras.callbacks.ModelCheckpoint(filepath="../../data_ga2/models/mobv2_m2_best.h5", monitor="val_loss", save_best_only=True)]
        model.fit(X_train, y, epochs=20, batch_size=32, validation_split=0.2, callbacks=callbacks)

        base_model.trainable = True
        model.compile(run_eagerly=True, optimizer=tf.keras.optimizers.Adam(1e-6), loss='binary_crossentropy', metrics=['binary_accuracy'])
        model.fit(X_train, y, epochs=20, batch_size=32, validation_split=0.2, callbacks=callbacks)
        self.model = model

    def predict(self, X):
        return self.model.predict(tf.keras.applications.mobilenet_v2.preprocess_input(X), verbose=0)

    def __call__(self, X):
        return self.predict(X)

In [107]:
mobv2_m2 = MobileNetV2_fine_Model(X_train, y_train)
make_sound()
# mobv2_m2.model.summary()

Epoch 1/20
19/19 [==============================] - 50s 3s/step - loss: 0.3812 - binary_accuracy: 0.9030 - val_loss: 0.2123 - val_binary_accuracy: 0.9370
Epoch 2/20
19/19 [==============================] - 52s 3s/step - loss: 0.0812 - binary_accuracy: 0.9751 - val_loss: 0.1731 - val_binary_accuracy: 0.9493
Epoch 3/20
19/19 [==============================] - 49s 3s/step - loss: 0.0232 - binary_accuracy: 0.9945 - val_loss: 0.1602 - val_binary_accuracy: 0.9547
Epoch 4/20
19/19 [==============================] - 39s 2s/step - loss: 0.0099 - binary_accuracy: 0.9996 - val_loss: 0.1678 - val_binary_accuracy: 0.9547
Epoch 5/20
19/19 [==============================] - 38s 2s/step - loss: 0.0062 - binary_accuracy: 1.0000 - val_loss: 0.1704 - val_binary_accuracy: 0.9527
Epoch 1/20
19/19 [==============================] - 43s 2s/step - loss: 0.0511 - binary_accuracy: 0.9804 - val_loss: 0.1679 - val_binary_accuracy: 0.9543
Epoch 2/20
19/19 [==============================] - 43s 2s/step - loss: 0.04

### 2.1.3. Parameters from scratch (0.9277)

In [19]:
class MobileNetV2_scratch_Model:

    def __init__(self):
        mobv2_model = tf.keras.applications.MobileNetV2(include_top=False, input_shape=(224,224,3), weights=None)
        new_l = tf.keras.layers.Flatten()(mobv2_model.output)
        new_l = tf.keras.layers.Dense(20, activation="sigmoid")(new_l)
        self.model = tf.keras.Model(inputs=mobv2_model.input, outputs=new_l)

    def fit(self, X, y):
        X_train = tf.keras.applications.mobilenet_v2.preprocess_input(X)
        self.model.compile(run_eagerly=True, optimizer=tf.keras.optimizers.Adam(1e-5), loss='binary_crossentropy', metrics=['binary_accuracy'])
        callbacks = [tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3, min_delta=0.01),
                     tf.keras.callbacks.ModelCheckpoint(filepath="../../data_ga2/models/mobv2_m3_best.h5", monitor="val_loss", save_best_only=True)]
        self.model.fit(X_train, y, epochs=30,
                       batch_size=32, validation_split=0.2, callbacks=callbacks)

    def predict(self, X):
        return self.model.predict(tf.keras.applications.mobilenet_v2.preprocess_input(X), verbose=0)

    def __call__(self, X):
        return self.predict(X)

In [20]:
mobv2_m3 = MobileNetV2_scratch_Model()
make_sound()
mobv2_m3.model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                           

In [21]:
history_mobv2_m3 = mobv2_m3.fit(X_train, y_train)
make_sound()

Epoch 1/30
19/19 [==============================] - 56s 3s/step - loss: 0.3962 - binary_accuracy: 0.8417 - val_loss: 0.6884 - val_binary_accuracy: 0.9277
Epoch 2/30
19/19 [==============================] - 55s 3s/step - loss: 0.2719 - binary_accuracy: 0.9278 - val_loss: 0.6828 - val_binary_accuracy: 0.9277
Epoch 3/30
19/19 [==============================] - 51s 3s/step - loss: 0.2687 - binary_accuracy: 0.9270 - val_loss: 0.6793 - val_binary_accuracy: 0.9277
Epoch 4/30
19/19 [==============================] - 46s 2s/step - loss: 0.2587 - binary_accuracy: 0.9270 - val_loss: 0.6749 - val_binary_accuracy: 0.9277
Epoch 5/30
19/19 [==============================] - 46s 2s/step - loss: 0.2571 - binary_accuracy: 0.9279 - val_loss: 0.6695 - val_binary_accuracy: 0.9277
Epoch 6/30
19/19 [==============================] - 46s 2s/step - loss: 0.2501 - binary_accuracy: 0.9284 - val_loss: 0.6640 - val_binary_accuracy: 0.9277
Epoch 7/30
19/19 [==============================] - 47s 2s/step - loss: 0.24

KeyboardInterrupt: 

## 2.2. VGG16
### 2.2.1. Transfer learning

In [18]:
X_train = reshape_images(train_df["img"], 224)
y_train = np.asarray(train_df[train_df.columns[:20]])
X_test = reshape_images(test_df["img"], 224)
print("Shape of X_train images is {} - Shape of X_test images is {} - Shape of y_train labels is {}".format(X_train[0].shape, X_test[0].shape, y_train.shape))

Shape of X_train images is (224, 224, 3) - Shape of X_test images is (224, 224, 3) - Shape of y_train labels is (749, 20)


In [19]:
class VGG16_Model:

    def __init__(self):
        vgg_model = tf.keras.applications.VGG16(include_top=False, input_shape=(224,224,3), weights="imagenet")
        for l in vgg_model.layers:
            l.trainable = False
        new_l = tf.keras.layers.Flatten()(vgg_model.output)
        new_l = tf.keras.layers.Dense(20, activation="sigmoid")(new_l)
        self.model = tf.keras.Model(inputs=vgg_model.input, outputs=new_l)

    def fit(self, X, y):
        X_train = tf.keras.applications.vgg16.preprocess_input(X)
        self.model.compile(run_eagerly=True, optimizer=tf.keras.optimizers.Adam(1e-5), loss='binary_crossentropy', metrics=['binary_accuracy'])
        callbacks = [tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3, min_delta=0.01),
                     tf.keras.callbacks.ModelCheckpoint(filepath="../../data_ga2/models/vgg_m1_best.h5", monitor="val_loss", save_best_only=True)]
        self.model.fit(X_train, y, epochs=20,
                       batch_size=32, validation_split=0.2, callbacks=callbacks)

    def predict(self, X):
        return self.model.predict(tf.keras.applications.vgg16.preprocess_input(X), verbose=0)

    def __call__(self, X):
        return self.predict(X)

In [20]:
vgg_m1 = VGG16_Model()
make_sound()
vgg_m1.model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [ ]:
history_vgg_m1 = vgg_m1.fit(X_train, y_train)
make_sound()

Epoch 1/20
19/19 [==============================] - 511s 27s/step - loss: 4.6859 - binary_accuracy: 0.5769 - val_loss: 3.4332 - val_binary_accuracy: 0.6467
Epoch 2/20
19/19 [==============================] - 529s 28s/step - loss: 2.8627 - binary_accuracy: 0.7053 - val_loss: 2.3076 - val_binary_accuracy: 0.7527
Epoch 3/20
19/19 [==============================] - 518s 27s/step - loss: 2.0692 - binary_accuracy: 0.7845 - val_loss: 1.8599 - val_binary_accuracy: 0.8140
Epoch 4/20
19/19 [==============================] - 526s 28s/step - loss: 1.6871 - binary_accuracy: 0.8309 - val_loss: 1.6676 - val_binary_accuracy: 0.8393
Epoch 5/20
19/19 [==============================] - 520s 27s/step - loss: 1.4731 - binary_accuracy: 0.8558 - val_loss: 1.5594 - val_binary_accuracy: 0.8550
Epoch 6/20
19/19 [==============================] - 523s 28s/step - loss: 1.3160 - binary_accuracy: 0.8743 - val_loss: 1.4853 - val_binary_accuracy: 0.8620
Epoch 7/20
19/19 [==============================] - 518s 27s/ste

### 2.2.2. Transfer learning + fine-tuning

In [ ]:
class VGG16_fine_Model:

    def __init__(self, X, y):
        base_model = tf.keras.applications.VGG16(include_top=False, input_shape=(224,224,3), weights="imagenet")
        base_model.trainable = False

        new_l = tf.keras.layers.Flatten()(base_model.output)
        new_l = tf.keras.layers.Dense(20, activation="sigmoid")(new_l)
        model = tf.keras.Model(base_model.input, new_l)

        X_train = tf.keras.applications.vgg16.preprocess_input(X)

        model.compile(run_eagerly=True, optimizer=tf.keras.optimizers.Adam(1e-4), loss='binary_crossentropy', metrics=['binary_accuracy'])
        callbacks = [tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3, min_delta=0.01),
                     tf.keras.callbacks.ModelCheckpoint(filepath="../../data_ga2/models/vgg_m2_best.h5", monitor="val_loss", save_best_only=True)]
        model.fit(X_train, y, epochs=20, batch_size=32, validation_split=0.2, callbacks=callbacks)

        base_model.trainable = True
        model.compile(run_eagerly=True, optimizer=tf.keras.optimizers.Adam(1e-6), loss='binary_crossentropy', metrics=['binary_accuracy'])
        model.fit(X_train, y, epochs=20, batch_size=32, validation_split=0.2, callbacks=callbacks)
        self.model = model

    def predict(self, X):
        return self.model.predict(tf.keras.applications.vgg16.preprocess_input(X), verbose=0)

    def __call__(self, X):
        return self.predict(X)

In [ ]:
vgg_m2 = VGG16_fine_Model(X_train, y_train)
make_sound()
vgg_m2.model.summary()

### 2.2.3. Parameters from scratch

In [ ]:
class VGG16_scratch_Model:

    def __init__(self):
        vgg_model = tf.keras.applications.MobileNetV2(include_top=False, input_shape=(224,224,3), weights=None)
        new_l = tf.keras.layers.Flatten()(vgg_model.output)
        new_l = tf.keras.layers.Dense(20, activation="sigmoid")(new_l)
        self.model = tf.keras.Model(inputs=vgg_model.input, outputs=new_l)

    def fit(self, X, y):
        X_train = tf.keras.applications.vgg16.preprocess_input(X)
        self.model.compile(run_eagerly=True, optimizer=tf.keras.optimizers.Adam(1e-5), loss='binary_crossentropy', metrics=['binary_accuracy'])
        callbacks = [tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=2, min_delta=0.01),
                     tf.keras.callbacks.ModelCheckpoint(filepath="../../data_ga2/models/vgg_m3_best.h5", monitor="val_loss", save_best_only=True)]
        self.model.fit(X_train, y, epochs=30,
                       batch_size=32, validation_split=0.2, callbacks=callbacks)

    def predict(self, X):
        return self.model.predict(tf.keras.applications.vgg16.preprocess_input(X), verbose=0)

    def __call__(self, X):
        return self.predict(X)

In [ ]:
vgg_m3 = VGG16_scratch_Model()
make_sound()
vgg_m3.model.summary()

In [ ]:
history_vgg_m3 = vgg_m3.fit(X_train, y_train)
make_sound()

## 2.3. InceptionV3 (0.9243/0.9583/0.9277)
### 2.3.1. Transfer learning (0.9243)

In [8]:
X_train = reshape_images(train_df["img"], 224)
y_train = np.asarray(train_df[train_df.columns[:20]])
X_test = reshape_images(test_df["img"], 224)
print("Shape of X_train images is {} - Shape of X_test images is {} - Shape of y_train labels is {}".format(X_train[0].shape, X_test[0].shape, y_train.shape))

Shape of X_train images is (224, 224, 3) - Shape of X_test images is (224, 224, 3) - Shape of y_train labels is (749, 20)


In [9]:
class InceptionV3_Model:

    def __init__(self):
        incv3_model = tf.keras.applications.InceptionV3(include_top=False, input_shape=(224,224,3), weights="imagenet")
        for l in incv3_model.layers:
            l.trainable = False
        new_l = tf.keras.layers.Flatten()(incv3_model.output)
        new_l = tf.keras.layers.Dense(20, activation="sigmoid")(new_l)
        self.model = tf.keras.Model(inputs=incv3_model.input, outputs=new_l)

    def fit(self, X, y):
        X_train = tf.keras.applications.inception_v3.preprocess_input(X)
        self.model.compile(run_eagerly=True, optimizer=tf.keras.optimizers.Adam(1e-5), loss='binary_crossentropy', metrics=['binary_accuracy'])
        callbacks = [tf.keras.callbacks.EarlyStopping(monitor="val_binary_accuracy", patience=3, min_delta=0.01),
                     tf.keras.callbacks.ModelCheckpoint(filepath="../../data_ga2/models/incv3_m1_best.h5", monitor="val_binary_accuracy", save_best_only=True)]
        self.model.fit(X_train, y, epochs=30,
                       batch_size=32, validation_split=0.2, callbacks=callbacks)

    def predict(self, X):
        return self.model.predict(tf.keras.applications.inception_v3.preprocess_input(X), verbose=0)

    def __call__(self, X):
        return self.predict(X)

In [11]:
incv3_m1 = InceptionV3_Model()
make_sound()
incv3_m1.model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_94 (Conv2D)             (None, 111, 111, 32  864         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_94 (BatchN  (None, 111, 111, 32  96         ['conv2d_94[0][0]']              
 ormalization)                  )                                                           

In [12]:
history_incv3_m1 = incv3_m1.fit(X_train, y_train)
make_sound()

Epoch 1/20
19/19 [==============================] - 100s 5s/step - loss: 0.4302 - binary_accuracy: 0.8258 - val_loss: 0.2979 - val_binary_accuracy: 0.9193
Epoch 2/20
19/19 [==============================] - 96s 5s/step - loss: 0.2833 - binary_accuracy: 0.9259 - val_loss: 0.2814 - val_binary_accuracy: 0.9217
Epoch 3/20
19/19 [==============================] - 96s 5s/step - loss: 0.2517 - binary_accuracy: 0.9277 - val_loss: 0.2557 - val_binary_accuracy: 0.9213
Epoch 4/20
19/19 [==============================] - 96s 5s/step - loss: 0.2221 - binary_accuracy: 0.9303 - val_loss: 0.2371 - val_binary_accuracy: 0.9243


### 2.3.2. Transfer learning + fine-tuning (0.9583)

In [13]:
class InceptionV3_fine_Model:

    def __init__(self, X, y):
        base_model = tf.keras.applications.InceptionV3(include_top=False, input_shape=(224,224,3), weights="imagenet")
        base_model.trainable = False

        new_l = tf.keras.layers.Flatten()(base_model.output)
        new_l = tf.keras.layers.Dense(20, activation="sigmoid")(new_l)
        model = tf.keras.Model(base_model.input, new_l)

        X_train = tf.keras.applications.inception_v3.preprocess_input(X)

        model.compile(run_eagerly=True, optimizer=tf.keras.optimizers.Adam(1e-4), loss='binary_crossentropy', metrics=['binary_accuracy'])
        callbacks = [tf.keras.callbacks.EarlyStopping(monitor="val_binary_accuracy", patience=3, min_delta=0.01),
                     tf.keras.callbacks.ModelCheckpoint(filepath="../../data_ga2/models/incv3_m2_best.h5", monitor="val_binary_accuracy", save_best_only=True)]
        model.fit(X_train, y, epochs=30, batch_size=32, validation_split=0.2, callbacks=callbacks)

        base_model.trainable = True
        model.compile(run_eagerly=True, optimizer=tf.keras.optimizers.Adam(1e-6), loss='binary_crossentropy', metrics=['binary_accuracy'])
        model.fit(X_train, y, epochs=30, batch_size=32, validation_split=0.2, callbacks=callbacks)
        self.model = model

    def predict(self, X):
        return self.model.predict(tf.keras.applications.inception_v3.preprocess_input(X), verbose=0)

    def __call__(self, X):
        return self.predict(X)

In [14]:
incv3_m2 = InceptionV3_fine_Model(X_train, y_train)
make_sound()
incv3_m2.model.summary()

Epoch 1/30
19/19 [==============================] - 92s 5s/step - loss: 0.3867 - binary_accuracy: 0.9025 - val_loss: 0.2104 - val_binary_accuracy: 0.9337
Epoch 2/30
19/19 [==============================] - 93s 5s/step - loss: 0.1208 - binary_accuracy: 0.9614 - val_loss: 0.1549 - val_binary_accuracy: 0.9503
Epoch 3/30
19/19 [==============================] - 92s 5s/step - loss: 0.0594 - binary_accuracy: 0.9824 - val_loss: 0.1450 - val_binary_accuracy: 0.9540
Epoch 4/30
19/19 [==============================] - 93s 5s/step - loss: 0.0326 - binary_accuracy: 0.9926 - val_loss: 0.1420 - val_binary_accuracy: 0.9570
Epoch 5/30
19/19 [==============================] - 92s 5s/step - loss: 0.0214 - binary_accuracy: 0.9966 - val_loss: 0.1394 - val_binary_accuracy: 0.9587
Epoch 1/30
19/19 [==============================] - 110s 6s/step - loss: 0.1213 - binary_accuracy: 0.9602 - val_loss: 0.1373 - val_binary_accuracy: 0.9593
Epoch 2/30
19/19 [==============================] - 114s 6s/step - loss: 0.

### 2.3.3. Parameters from scratch (0.9277)

In [15]:
class InceptionV3_scratch_Model:

    def __init__(self):
        incv3_model = tf.keras.applications.InceptionV3(include_top=False, input_shape=(224,224,3), weights=None)
        new_l = tf.keras.layers.Flatten()(incv3_model.output)
        new_l = tf.keras.layers.Dense(20, activation="sigmoid")(new_l)
        self.model = tf.keras.Model(inputs=incv3_model.input, outputs=new_l)

    def fit(self, X, y):
        X_train = tf.keras.applications.inception_v3.preprocess_input(X)
        self.model.compile(run_eagerly=True, optimizer=tf.keras.optimizers.Adam(1e-5), loss='binary_crossentropy', metrics=['binary_accuracy'])
        callbacks = [tf.keras.callbacks.EarlyStopping(monitor="val_binary_accuracy", patience=3, min_delta=0.01),
                     tf.keras.callbacks.ModelCheckpoint(filepath="../../data_ga2/models/incv3_m3_best.h5", monitor="val_binary_accuracy", save_best_only=True)]
        self.model.fit(X_train, y, epochs=30,
                       batch_size=32, validation_split=0.2, callbacks=callbacks)

    def predict(self, X):
        return self.model.predict(tf.keras.applications.inception_v3.preprocess_input(X), verbose=0)

    def __call__(self, X):
        return self.predict(X)

In [16]:
incv3_m3 = InceptionV3_scratch_Model()
make_sound()
incv3_m3.model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_282 (Conv2D)            (None, 111, 111, 32  864         ['input_4[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_282 (Batch  (None, 111, 111, 32  96         ['conv2d_282[0][0]']             
 Normalization)                 )                                                           

In [17]:
history_incv3_m3 = incv3_m3.fit(X_train, y_train)
make_sound()

Epoch 1/30
19/19 [==============================] - 101s 5s/step - loss: 0.3930 - binary_accuracy: 0.8513 - val_loss: 0.6241 - val_binary_accuracy: 0.9277
Epoch 2/30
19/19 [==============================] - 102s 5s/step - loss: 0.2608 - binary_accuracy: 0.9282 - val_loss: 0.5379 - val_binary_accuracy: 0.9277
Epoch 3/30
19/19 [==============================] - 99s 5s/step - loss: 0.2497 - binary_accuracy: 0.9280 - val_loss: 0.4900 - val_binary_accuracy: 0.9277
Epoch 4/30
19/19 [==============================] - 99s 5s/step - loss: 0.2424 - binary_accuracy: 0.9282 - val_loss: 0.4439 - val_binary_accuracy: 0.9277


## 2.4. MobileNet
### 2.4.1. Transfer learning

In [7]:
X_train = reshape_images(train_df["img"], 224)
y_train = np.asarray(train_df[train_df.columns[:20]])
X_test = reshape_images(test_df["img"], 224)
print("Shape of X_train images is {} - Shape of X_test images is {} - Shape of y_train labels is {}".format(X_train[0].shape, X_test[0].shape, y_train.shape))

Shape of X_train images is (224, 224, 3) - Shape of X_test images is (224, 224, 3) - Shape of y_train labels is (749, 20)


In [8]:
class MobileNet_Model:

    def __init__(self):
        mob_model = tf.keras.applications.MobileNet(include_top=False, input_shape=(224,224,3), weights="imagenet")
        for l in mob_model.layers:
            l.trainable = False
        new_l = tf.keras.layers.Flatten()(mob_model.output)
        new_l = tf.keras.layers.Dense(20, activation="sigmoid")(new_l)
        self.model = tf.keras.Model(inputs=mob_model.input, outputs=new_l)

    def fit(self, X, y):
        X_train = tf.keras.applications.mobilenet.preprocess_input(X)
        self.model.compile(run_eagerly=True, optimizer=tf.keras.optimizers.Adam(1e-5), loss='binary_crossentropy', metrics=['binary_accuracy'])
        callbacks = [tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3, min_delta=0.01),
                     tf.keras.callbacks.ModelCheckpoint(filepath="../../data_ga2/models/mob_m1_best.h5", monitor="val_loss", save_best_only=True)]
        self.model.fit(X_train, y, epochs=30,
                       batch_size=32, validation_split=0.2, callbacks=callbacks)

    def predict(self, X):
        return self.model.predict(tf.keras.applications.mobilenet.preprocess_input(X), verbose=0)

    def __call__(self, X):
        return self.predict(X)

In [9]:
mob_m1 = MobileNet_Model()
make_sound()
mob_m1.model.summary()

Exception: URL fetch failure on https://storage.googleapis.com/tensorflow/keras-applications/mobilenet/mobilenet_1_0_224_tf_no_top.h5: None -- [Errno 11001] getaddrinfo failed

In [ ]:
history_mob_m1 = mob_m1.fit(X_train, y_train)
make_sound()

### 2.4.2. Transfer learning + fine-tuning

In [ ]:
class MobileNet_fine_Model:

    def __init__(self, X, y):
        base_model = tf.keras.applications.MobileNet(include_top=False, input_shape=(224,224,3), weights="imagenet")
        base_model.trainable = False

        new_l = tf.keras.layers.Flatten()(base_model.output)
        new_l = tf.keras.layers.Dense(20, activation="sigmoid")(new_l)
        model = tf.keras.Model(base_model.input, new_l)

        X_train = tf.keras.applications.mobilenet.preprocess_input(X)

        model.compile(run_eagerly=True, optimizer=tf.keras.optimizers.Adam(1e-4), loss='binary_crossentropy', metrics=['binary_accuracy'])
        callbacks = [tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3, min_delta=0.01),
                     tf.keras.callbacks.ModelCheckpoint(filepath="../../data_ga2/models/mob_m2_best.h5", monitor="val_loss", save_best_only=True)]
        model.fit(X_train, y, epochs=30, batch_size=32, validation_split=0.2, callbacks=callbacks)

        base_model.trainable = True
        model.compile(run_eagerly=True, optimizer=tf.keras.optimizers.Adam(1e-6), loss='binary_crossentropy', metrics=['binary_accuracy'])
        model.fit(X_train, y, epochs=30, batch_size=32, validation_split=0.2, callbacks=callbacks)
        self.model = model

    def predict(self, X):
        return self.model.predict(tf.keras.applications.mobilenet.preprocess_input(X), verbose=0)

    def __call__(self, X):
        return self.predict(X)

In [ ]:
mob_m2 = MobileNet_fine_Model(X_train, y_train)
make_sound()
# mob_m2.model.summary()

### 2.4.3. Parameters from scratch

In [28]:
class MobileNet_scratch_Model:

    def __init__(self):
        mob_model = tf.keras.applications.MobileNet(include_top=False, input_shape=(224,224,3), weights=None)
        new_l = tf.keras.layers.Flatten()(mob_model.output)
        new_l = tf.keras.layers.Dense(20, activation="sigmoid")(new_l)
        self.model = tf.keras.Model(inputs=mob_model.input, outputs=new_l)

    def fit(self, X, y):
        X_train = tf.keras.applications.mobilenet.preprocess_input(X)
        self.model.compile(run_eagerly=True, optimizer=tf.keras.optimizers.Adam(1e-5), loss='binary_crossentropy', metrics=['binary_accuracy'])
        callbacks = [tf.keras.callbacks.EarlyStopping(monitor="val_binary_accuracy", patience=2, min_delta=0.01),
                     tf.keras.callbacks.ModelCheckpoint(filepath="../../data_ga2/models/mob_m3_best.h5", monitor="val_binary_accuracy", save_best_only=True)]
        return self.model.fit(X_train, y, epochs=30,
                       batch_size=32, validation_split=0.25, callbacks=callbacks)

    def predict(self, X):
        return self.model.predict(tf.keras.applications.mobilenet.preprocess_input(X), verbose=0)

    def __call__(self, X):
        return self.predict(X)

In [29]:
mob_m3 = MobileNet_scratch_Model()
make_sound()
mob_m3.model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                                    

In [30]:
history_mob_m3 = mob_m3.fit(X_train, y_train)
make_sound()

Epoch 1/30
18/18 [==============================] - 43s 2s/step - loss: 0.4060 - binary_accuracy: 0.8367 - val_loss: 0.6506 - val_binary_accuracy: 0.9285
Epoch 2/30
18/18 [==============================] - 42s 2s/step - loss: 0.2700 - binary_accuracy: 0.9266 - val_loss: 0.5932 - val_binary_accuracy: 0.9285
Epoch 3/30
18/18 [==============================] - 42s 2s/step - loss: 0.2689 - binary_accuracy: 0.9283 - val_loss: 0.5549 - val_binary_accuracy: 0.9285


TypeError: 'History' object is not iterable

## 2.5. By hand

In [17]:
class Basic_Model:

    def __init__(self):
        model  = tf.keras.models.Sequential()
        model.add(tf.keras.layers.Conv2D(None, kernel_size=(3,3), input_shape=(244,244,3)))
        model.add(tf.keras.layers.MaxPooling2D())
        model.add(tf.keras.layers.Dense(224, activation="relu"))
        model.add(tf.keras.layers.Flatten())
        model.add(tf.keras.layers.Dense(20, activation="sigmoid"))
        self.model = model

    def fit(self, X_train, y):
        self.model.compile(run_eagerly=True, optimizer=tf.keras.optimizers.Adam(1e-5), loss='binary_crossentropy', metrics=['binary_accuracy'])
        callbacks = [tf.keras.callbacks.EarlyStopping(monitor="val_binary_accuracy", patience=3, min_delta=0.01),
                     tf.keras.callbacks.ModelCheckpoint(filepath="../../data_ga2/models/basic_best.h5", monitor="val_binary_accuracy", save_best_only=True)]
        self.model.fit(X_train, y, epochs=30,
                       batch_size=32, validation_split=0.25, callbacks=callbacks)

    def predict(self, X):
        return self.model.predict(X, verbose=0)

    def __call__(self, X):
        return self.predict(X)

In [18]:
bas_m1 = Basic_Model()
make_sound()
bas_m1.model.summary()

TypeError: unsupported operand type(s) for *: 'NoneType' and 'int'